<a href="https://colab.research.google.com/github/prof-atritiack/inteli-class/blob/main/Aula_INTELI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. INSTALAÇÃO DE DEPENDÊNCIAS E IMPORTAÇÃO DOS PACOTES**

In [ ]:
# Vamos inicialmente instalar as dependências para podermos criar nossa API
# FastAPI ----> framework WEB para criarmos a API do modelo.
# Uvicorn ----> nosso servidor HTTP para rodar o FastAPI.
# Pyngrok ----> nosso túnel para conectar a API que vamos criar ao mundo...
# Esse recurso cria um link público que o NodeRED vai acessar
!pip -q install fastapi uvicorn pyngrok "pydantic<2"

In [ ]:
# Pandas nos ajudará com a manipulação dos dados
import pandas as pd
import numpy as np
# A função train_test_split facilita a separação de dados de treino e teste
from sklearn.model_selection import train_test_split
# Utilizaremos o algoritmo Logistic Regression para treinarmos um modelo de classificação
from sklearn.linear_model import LogisticRegression
# Ferramentas para obtenção das métricas do modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score
# joblib nos facilita a serialização do modelo
import joblib
# Oculta avisos nos outputs
import warnings
warnings.filterwarnings('ignore')

from pydantic import BaseModel

from google.colab import userdata

from pyngrok import ngrok
import nest_asyncio, threading
import uvicorn

from fastapi import FastAPI

## **2. IMPORTAÇÃO DOS DADOS**

In [ ]:
dados = pd.read_csv('/content/motor_min_sintetico.csv')

## **3. ANÁLISE EXPLORATÓRIA**

In [ ]:
# Dimensões do dataset
print(f'O conjunto de dados tem {dados.shape[0]} registros e {dados.shape[1]} atributos.')

O conjunto de dados tem 3000 registros e 6 atributos.


In [ ]:
# Relação de atributos (colunas)
print(f'Os atributos do conjunto de dados são: {list(dados.columns)}')

Os atributos do conjunto de dados são: ['timestamp', 'device_id', 'motor_temp', 'vibration_rms', 'current', 'alerta']


In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp      3000 non-null   object 
 1   device_id      3000 non-null   object 
 2   motor_temp     3000 non-null   float64
 3   vibration_rms  3000 non-null   float64
 4   current        3000 non-null   float64
 5   alerta         3000 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 140.8+ KB


In [ ]:
# Primeiros registros
dados.head()

In [ ]:
dados.describe()

,motor_temp,vibration_rms,current,alerta
count,3000.000000,3000.000000,3000.000000,3000.00000
mean,71.160573,2.641240,24.180490,0.10000
std,7.126848,0.759162,3.401169,0.30005
min,43.560000,0.735000,14.210000,0.00000
25%,66.420000,2.216500,21.980000,0.00000
50%,70.765000,2.537000,23.990000,0.00000
75%,75.052500,2.945000,26.182500,0.00000
max,106.740000,7.953000,43.190000,1.00000


In [ ]:
# Contagem de classes da coluna alvo (alerta)
# 0: normal
# 1: anomalia
dados['alerta'].value_counts()

,count
alerta,
0,2700
1,300


In [ ]:
# Proporção de anomalias
print(f'Proporção de anomalias:', round(dados['alerta'].mean() * 100, 2), '%')

Proporção de anomalias: 10.0 %


## **4. DEFINIÇÃO DE FEATURES E TARGET / SEPARAÇÃO DE DADOS DE TREINO E TESTE**

In [ ]:
# Features
X = dados[['motor_temp', 'vibration_rms', 'current']]

In [ ]:
# Target
y = dados['alerta']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
# Stratify ---> garante o equilíbrio de classes nos sets de treino e teste.

## **5. CRIAÇÃO, TREINAMENTO E AVALIAÇÃO DO MODELO PREDITIVO**

In [ ]:
# Instanciando o modelo
modeloLR = LogisticRegression()

In [ ]:
# Treinando o modelo
modeloLR.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# Fazendo previsões
y_predict = modeloLR.predict(X_test)

In [ ]:
# Avaliar métricas principais
# Acurácia: Percentual total de acertos do modelo.
# Exemplo: De todas as previsões, quantas estavam corretas?
acuracia = accuracy_score(y_test, y_predict)
print(f'Acurácia : {acuracia:.3f} | O modelo acertou {int(acuracia * 100)}% do total de casos testados.')

Acurácia : 0.960 | O modelo acertou 96% do total de casos testados.


In [ ]:
# Precisão: Mede a confiança nos alertas.
# Exemplo: Dos casos que o modelo marcou como anomalia, quantos realmente eram?
# Alta precisão ---> poucos alarmes falsos
precisao = precision_score(y_test, y_predict)
print(f'Precisão : {precisao:.3f} | Dos alertas gerados, {int(precisao * 100)}% eram realmente anomalias')

Precisão : 0.860 | Dos alertas gerados, 86% eram realmente anomalias


In [ ]:
# Recall (revocação / sensibilidade): Mede a capacidade de detectar problemas reais.
# Exemplo: De todas as anomalias existentes, quantas o modelo conseguiu achar?
# Alto recall ---> poucos falsos negativos
recall = recall_score(y_test, y_predict)
print(f'Revocação: {recall:.3f} | Das anomalias reais do set de teste, o modelo detectou {int(recall * 100)}%')

Revocação: 0.717 | Das anomalias reais do set de teste, o modelo detectou 71%


## **6. SALVANDO O MODELO E TESTANDO UMA PREVISÃO**

In [ ]:
# Salva o modelo em arquivo binário (.pkl)
modelo_salvo = joblib.dump(modeloLR, "modelo_motor.pkl")

In [ ]:
# Carrega o modelo salvo
modelo_carregado = joblib.load("modelo_motor.pkl")

In [ ]:
# Caso normal (motor operando bem)
teste_normal = [[65, 2.5, 22]]  # temp, vibração, corrente
# Caso de anomalia (motor superaquecido e com alta corrente)
teste_alarme = [[110, 5.0, 40]]

In [ ]:
# Fazer previsões
pred_normal = modelo_carregado.predict(teste_normal)[0]
print("\nCaso Normal:")
print("{ Predição:", "ALERTA" if pred_normal == 1 else "OK", "}")


Caso Normal:
{ Predição: OK }


In [ ]:
pred_alarme = modelo_carregado.predict(teste_alarme)[0]
print("\nCaso de Anomalia:")
print("{ Predição:", "ALERTA" if pred_alarme == 1 else "OK", "}")


Caso de Anomalia:
{ Predição: ALERTA }


## **7. CRIANDO A API COM FASTAPI**

In [ ]:
# Criar a aplicação
app = FastAPI(title="API - Modelo de Anomalias do Motor")

In [ ]:
# Classe de entrada (validação automática com Pydantic)
class Entrada(BaseModel):
    motor_temp: float
    vibration_rms: float
    current: float

In [ ]:
# Rota principal
@app.post("/predict")
def predict(data: Entrada):
    # Converter para formato esperado pelo modelo
    X = np.array([[data.motor_temp, data.vibration_rms, data.current]])
    pred = modelo_carregado.predict(X)[0]
    return {
        "pred": "ALERTA" if pred == 1 else "OK",
        "dados_recebidos": data.dict()
    }

In [ ]:
# Fazer o deploy da API
token = userdata.get('ngrok_token')
ngrok.set_auth_token(token)

In [ ]:
# Aplicar patch para o Colab
nest_asyncio.apply()

In [ ]:
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="warning")

# sobe o servidor em background
threading.Thread(target=run, daemon=True).start()

# cria o túnel público
public_url = ngrok.connect(8000, "http")
print("Docs:", public_url.public_url + "/docs")
print("Predict:", public_url.public_url + "/predict")

Docs: https://tangentially-untreasurable-coralee.ngrok-free.dev/docs
Predict: https://tangentially-untreasurable-coralee.ngrok-free.dev/predict


In [ ]:
!curl -s -X POST https://SEU_NGROK/predict \
  -H "Content-Type: application/json" \
  -d '{"motor_temp":85,"vibration_rms":4.2,"current":30}'
